In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import LabelEncoder
from kmodes.kprototypes import KPrototypes
from matplotlib import style
from sklearn.preprocessing import MinMaxScaler

Import data

In [102]:
data = pd.read_csv('cleaned.csv')

c:\users\anupi\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (1,12,19,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Copy the data in dataframe df

In [103]:
df=data.copy()
#df.head()

Keep data only for the year 2008 and 2009 (study period).

In [104]:
start=datetime.datetime(2008, 1, 1)
end=datetime.datetime(2009, 12, 31)

In [105]:
df['APPL_DATE']= pd.to_datetime(df['APPL_DATE'],dayfirst=True)
indices=[]

df = df[df['APPL_DATE']>start]
df = df[df['APPL_DATE']<end]

#len(df)

Remove columns which we don't want our model to take

In [106]:
df.drop(['OD_AMT','ACTIVITY_NAME','CLIENT_SRLNO','UNIT_NAME','APPL_DATE','PRODUCT_NAME','PIN_CODE','REGIS_DATE','ASSET_BASE','VILLAGE_NAME','REPAY_TYPE','Unnamed: 0','FAMILY_MEN','FAMILY_WOM','NO_OF_INSTALL','CITY','ACTIVE_YN','TOTAL_INSTALLMENT_PAID'],axis=1,inplace=True)
#len(df)

Selecting only the required district

In [107]:
df = df[df['UNIT_CODE']==31]
#len(df)

Remove NaN's and convert values in the data to str or int

In [108]:
df.dropna(inplace=True)
df = df.astype({"VILLAGE_CODE": str,'FAMILY_MEM':int,'GUARANTOR':str,'CO_OBLIGANT':str,'CO_LATERAL':str,'CATEGORY':str,'GENDER':str})
#len(df)
#df.head()

for deault column

In [109]:
df['Default'] = (df['ODDAYS'] > 0).astype(int)

In [110]:
#df.head()

In [111]:
df.drop('ODDAYS',inplace=True,axis=1)
df.reset_index(inplace=True)

In [112]:
df.drop('index',inplace=True,axis=1)
#df.head()

In [113]:
#df.describe()

Making bins for numerical variables

In [114]:
bins = pd.IntervalIndex.from_tuples([(0, 10000), (10000, 12000),(12000, 15000),(15000,200000)])
df['APPL_AMT_CAT']=pd.cut(df['APPL_AMT'], bins)

In [115]:
bins = pd.IntervalIndex.from_tuples([(0, 2), (2, 5), (5, 100)])
df['FAMILY_MEM_CAT']=pd.cut(df['FAMILY_MEM'], bins)

In [116]:
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 800),(800, 1000),(1000,200000)])
df['REGULAR_CAT']=pd.cut(df['REGULAR'], bins)

In [117]:
bins = pd.IntervalIndex.from_tuples([(0, 950), (950, 1500),(1500, 2000),(2000,200000)])
df['FESTIVE_CAT']=pd.cut(df['FESTIVE'], bins)

In [118]:
bins = pd.IntervalIndex.from_tuples([(0, 25), (25, 35),(35, 45),(45,150)])
df['AGE_CAT']=pd.cut(df['AGE'], bins)

In [119]:
#list(df.columns)

In [120]:
df.head()

,CLIENT_CODE,LOAN_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,...,CATEGORY,AGE,GENDER,ACCOMADATION,Default,APPL_AMT_CAT,FAMILY_MEM_CAT,REGULAR_CAT,FESTIVE_CAT,AGE_CAT
0,31006642,9935635,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,...,O.B.C,32.0,F,R,0,"(0, 10000]","(2, 5]","(800, 1000]","(2000, 200000]","(25, 35]"
1,31006645,9946982,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,...,O.B.C,27.0,F,R,1,"(15000, 200000]","(5, 100]","(800, 1000]","(1500, 2000]","(25, 35]"
2,31006646,9946986,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,...,O.B.C,41.0,F,R,1,"(15000, 200000]","(5, 100]","(800, 1000]","(1500, 2000]","(35, 45]"
3,31006652,9946985,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,...,O.B.C,31.0,F,R,1,"(15000, 200000]","(5, 100]","(800, 1000]","(1500, 2000]","(25, 35]"
4,31006653,9946983,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,...,O.B.C,32.0,F,R,1,"(15000, 200000]","(2, 5]","(0, 500]","(950, 1500]","(25, 35]"


Encode the categorical variables. 
Firct convert the dataframe in a numpy array and then choose the columns to encode.

In [121]:
labelencoder1=LabelEncoder()

In [122]:
#print(list(df.columns))

In [123]:
column_head=list(df.columns)
cat_var_index=[3,5,6,8,11,12,13,14,15,17,18,20,21,22,23,24]
cat_var=[]
for i in range (len(cat_var_index)):
    cat_var.append(column_head[cat_var_index[i]])


In [124]:
X=df.iloc[:,cat_var_index].values
#X.shape

In [125]:
for i in range (0,16):
        X[:,i]=labelencoder1.fit_transform(X[:,i])
        #print (i)

In [126]:
#df.reset_index(inplace=True)
#df.drop('index',axis=1,inplace=True)
#print(df.head())

In [127]:
#df.head()

In [128]:
df2=df.copy()

ID is the identifier for each entry in out data

In [129]:
CLIENT_CODE = df['CLIENT_CODE'].to_list()
CLIENT_CODE= np.asarray(CLIENT_CODE)
#len(ID)

In [130]:
#print(X.shape)

In [131]:
df3=pd.DataFrame(X)

In [132]:
df3.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,5,25,5,4,0,0,0,0,1,0,1,0,1,2,3,1
1,2,6,108,4,0,0,0,0,1,0,1,3,2,2,2,1
2,2,6,108,4,0,0,0,0,1,0,1,3,2,2,2,2
3,3,13,108,3,0,0,0,0,1,0,1,3,2,2,2,1
4,2,6,108,4,0,0,0,0,1,0,1,3,1,0,1,1


In [133]:
df4=pd.DataFrame()
for i in range (len(cat_var)):
    df4[cat_var[i]]=df[cat_var[i]]
    df4[i]=df3[i]

In [134]:
df4.to_csv('categoryVSencoding.csv')

Applying KModes algorithm to the data stored in X

In [135]:
from kmodes.kmodes import KModes

In [136]:
n_clust=10
km = KModes(n_clusters=n_clust, init='Huang', n_init=2, verbose=1)
clusters = km.fit_predict(X)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 1421, cost: 19270.0
Run 1, iteration: 2/100, moves: 54, cost: 19270.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 1700, cost: 18913.0
Run 2, iteration: 2/100, moves: 406, cost: 18886.0
Run 2, iteration: 3/100, moves: 47, cost: 18886.0
Best run was number 2


Location of the centroids

In [137]:
print(km.cluster_centroids_)

[[  2   6  62   3   0   0   0   0   1   0   0   2   0   0   0   0]
 [  3   6  83   3   0   0   0   0   0   0   1   2   1   3   3   1]
 [  3  13  97   4   0   0   0   0   1   1   1   2   1   0   0   2]
 [  3  26  47   3   0   0   0   0   1   0   0   0   0   0   0   1]
 [  3  21  47   4   0   0   0   0   0   0   1   0   0   2   1   2]
 [  3  13  78   4   0   0   0   0   1   1   1   3   1   1   1   1]
 [  3  21 106   3   0   0   0   0   1   0   1   0   0   2   2   1]
 [  4  21  84   3   0   0   0   0   1   0   0   0   0   0   0   1]
 [  2   6  75   3   0   0   0   0   1   1   1   3   1   1   1   0]
 [  3  13  62   3   0   0   0   0   1   0   1   2   0   2   1   1]]


Save the obtained clusters in a dictionary where keys are ID's and values are the cluster numbers. Also, append the cluster numbers to original dataframe.

In [138]:
clust_dict={}
clust=[]
for s, c in zip(CLIENT_CODE, clusters):
    clust_dict[s]=c
    clust.append(c)
df2['cluster']=clust
df3['cluster']=clust

In [139]:
#df3.head(25)

Calculating the distance of each point from its cluster

In [140]:
distance=[]
for i in range (len(df3)):
    count=0
    cl=df3.get_value(i, 'cluster')
    #print(cl)
    if i%500==0:
        print(i)
    for j in range(0,16):
        if df3.iloc[i][j]!=km.cluster_centroids_[cl][j]:
            count=count+1
    distance.append(count)
        

c:\users\anupi\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


0
500
1000
1500
2000
2500
3000
3500
4000


In [141]:
df2['distance']=distance
df3['distance']=distance

In [142]:
df2.to_csv('KMODES_IDvsClustvsDist_UCode=31.csv')
df3.to_csv('KMODES_onlyCatVar_IDvsClustvsDist_UCode=31.csv')

Summary statistics

In [143]:
for i in range(0,n_clust):
    print ("Cluster",i,":", df2.groupby('cluster').count().iloc[i]['CLIENT_CODE'])
#df.groupby('cluster').count()

Cluster 0 : 424
Cluster 1 : 594
Cluster 2 : 359
Cluster 3 : 657
Cluster 4 : 356
Cluster 5 : 473
Cluster 6 : 444
Cluster 7 : 382
Cluster 8 : 328
Cluster 9 : 172


In [144]:
summary = pd.read_csv('KMODES_IDvsClustvsDist_UCode=31.csv')

In [145]:
summary.groupby('cluster').count()

,Unnamed: 0,CLIENT_CODE,LOAN_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,...,AGE,GENDER,ACCOMADATION,Default,APPL_AMT_CAT,FAMILY_MEM_CAT,REGULAR_CAT,FESTIVE_CAT,AGE_CAT,distance
cluster,,,,,,,,,,,,,,,,,,,,,
0,424,424,424,424,424,424,424,424,424,424,...,424,424,424,424,424,424,424,424,424,424
1,594,594,594,594,594,594,594,594,594,594,...,594,594,594,594,594,594,594,594,594,594
2,359,359,359,359,359,359,359,359,359,359,...,359,359,359,359,359,359,359,359,359,359
3,657,657,657,657,657,657,657,657,657,657,...,657,657,657,657,657,657,657,657,657,657
4,356,356,356,356,356,356,356,356,356,356,...,356,356,356,356,356,356,356,356,356,356
5,473,473,473,473,473,473,473,473,473,473,...,473,473,473,473,473,473,473,473,473,473
6,444,444,444,444,444,444,444,444,444,444,...,444,444,444,444,444,444,444,444,444,444
7,382,382,382,382,382,382,382,382,382,382,...,382,382,382,382,382,382,382,382,382,382
8,328,328,328,328,328,328,328,328,328,328,...,328,328,328,328,328,328,328,328,328,328


In [146]:
summary.drop('Unnamed: 0',inplace=True,axis=1)

Based on category

In [147]:
print('Clusterwise category for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].index.tolist())
        print(count)
        
df1 = summary.groupby('CATEGORY').count()
df1.reset_index(inplace=True)
#df1.head()
df1 = df1[['CATEGORY','CLIENT_CODE']].copy()

print('Category-wise summary for all clusters:\n')
for cat in summary['CATEGORY'].unique().tolist():
    print('\nFor category',cat,'the total number of applications are-',df1[df1['CATEGORY']==cat]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].tolist())
        cats = summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].index.tolist()
        df_dummy = pd.DataFrame({'CAT':cats,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['CAT']==cat]['count']*100/float(df1[df1['CATEGORY']==cat]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise category for all clusters

Cluster 0 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[60.14, 16.51, 13.92, 8.25, 1.18]
Cluster 1 :-
['MINORITY', 'O.B.C', 'S.C', 'S.T', 'O.C']
[45.62, 23.57, 19.02, 9.76, 2.02]
Cluster 2 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[55.71, 17.55, 15.04, 9.75, 1.95]
Cluster 3 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[59.67, 24.66, 10.2, 3.96, 1.52]
Cluster 4 :-
['MINORITY', 'S.C', 'O.B.C', 'S.T', 'O.C']
[42.98, 24.44, 23.6, 8.43, 0.56]
Cluster 5 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[60.89, 16.07, 14.38, 6.77, 1.9]
Cluster 6 :-
['O.B.C', 'S.C', 'S.T', 'MINORITY', 'O.C']
[63.74, 22.07, 6.98, 6.31, 0.9]
Cluster 7 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[40.84, 28.8, 17.8, 11.78, 0.79]
Cluster 8 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[70.73, 12.8, 7.93, 5.79, 2.74]
Cluster 9 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[75.58, 9.3, 7.56, 4.65, 2.91]
Category-wise summary for all clusters:


For category O.B.C the total n

Based on default behaviour

In [148]:
print('Clusterwise defualt behaviour\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['Default'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['Default'].value_counts()[:2].index.tolist())
        print(count)
        
df1 = summary.groupby('Default').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['Default','CLIENT_CODE']].copy()

print('Default behaviour wise summary for all clusters:\n')
for default in summary['Default'].unique().tolist():
    print('\nFor default',default,'the total number of applications are-',df1[df1['Default']==default]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['Default'].value_counts()[:2].tolist())
        defs = summary[summary['cluster']==i]['Default'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Def':defs,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Def']==default]['count']*100/float(df1[df1['Default']==default]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise defualt behaviour

Cluster 0 :-
[1, 0]
[54.01, 45.99]
Cluster 1 :-
[0, 1]
[68.35, 31.65]
Cluster 2 :-
[0, 1]
[68.25, 31.75]
Cluster 3 :-
[1, 0]
[50.68, 49.32]
Cluster 4 :-
[0, 1]
[53.09, 46.91]
Cluster 5 :-
[0, 1]
[72.73, 27.27]
Cluster 6 :-
[0, 1]
[54.05, 45.95]
Cluster 7 :-
[1, 0]
[68.06, 31.94]
Cluster 8 :-
[0, 1]
[67.38, 32.62]
Cluster 9 :-
[0, 1]
[62.21, 37.79]
Default behaviour wise summary for all clusters:


For default 0 the total number of applications are-  2393
Cluster 0 has below %
1    8.148767
Name: count, dtype: float64
Cluster 1 has below %
0    16.966151
Name: count, dtype: float64
Cluster 2 has below %
0    10.238195
Name: count, dtype: float64
Cluster 3 has below %
1    13.53949
Name: count, dtype: float64
Cluster 4 has below %
0    7.898036
Name: count, dtype: float64
Cluster 5 has below %
0    14.375261
Name: count, dtype: float64
Cluster 6 has below %
0    10.029252
Name: count, dtype: float64
Cluster 7 has below %
1    5.098203
Name: count, dtype: fl

Based on Secured/Unsecured status

In [149]:
print('Clusterwise secured/unsecured for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:5].index.tolist())
        print(count)
        
df1 = summary.groupby('UNSECURE_SECURE').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['UNSECURE_SECURE','CLIENT_CODE']].copy()


print('Secured/unsecured wise summary for all clusters:\n')
for status in summary['UNSECURE_SECURE'].unique().tolist():
    print('\nFor status',status,'the total number of applications are-',df1[df1['UNSECURE_SECURE']==status]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:2].tolist())
        statuses = summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'STATUS':statuses,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['STATUS']==status]['count']*100/float(df1[df1['UNSECURE_SECURE']==status]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise secured/unsecured for all clusters

Cluster 0 :-
['U']
[100.0]
Cluster 1 :-
['U']
[100.0]
Cluster 2 :-
['U']
[100.0]
Cluster 3 :-
['U']
[100.0]
Cluster 4 :-
['U']
[100.0]
Cluster 5 :-
['U']
[100.0]
Cluster 6 :-
['U']
[100.0]
Cluster 7 :-
['U']
[100.0]
Cluster 8 :-
['U']
[100.0]
Cluster 9 :-
['U']
[100.0]
Secured/unsecured wise summary for all clusters:


For status U the total number of applications are-  4189
Cluster 0 has below %
0    10.121747
Name: count, dtype: float64
Cluster 1 has below %
0    14.179995
Name: count, dtype: float64
Cluster 2 has below %
0    8.570064
Name: count, dtype: float64
Cluster 3 has below %
0    15.683934
Name: count, dtype: float64
Cluster 4 has below %
0    8.498448
Name: count, dtype: float64
Cluster 5 has below %
0    11.291478
Name: count, dtype: float64
Cluster 6 has below %
0    10.599188
Name: count, dtype: float64
Cluster 7 has below %
0    9.119122
Name: count, dtype: float64
Cluster 8 has below %
0    7.830031
Name: count, dtype: f

In [150]:
#Based on house type
print('Clusterwise house type for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].index.tolist())
        print(count)        
        
df1 = summary.groupby('HOUSE_TYPE').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['HOUSE_TYPE','CLIENT_CODE']].copy()


print('House type wise summary for all clusters:\n')
for htype in summary['HOUSE_TYPE'].unique().tolist():
    print('\nFor house type',htype,'the total number of applications are-',df1[df1['HOUSE_TYPE']==htype]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].tolist())
        types = summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].index.tolist()
        df_dummy = pd.DataFrame({'HOUSE':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['HOUSE']==htype]['count']*100/float(df1[df1['HOUSE_TYPE']==htype]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise house type for all clusters

Cluster 0 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets']
[72.88, 24.76, 2.12, 0.24]
Cluster 1 :-
['Tatched', 'Tiles', 'Sheets', 'R.C.C', 'Stones']
[65.66, 24.92, 4.71, 4.55, 0.17]
Cluster 2 :-
['Tiles', 'Tatched', 'R.C.C', 'Sheets', 'Stones']
[70.47, 25.07, 2.79, 1.39, 0.28]
Cluster 3 :-
['Tatched', 'Tiles', 'R.C.C']
[78.08, 21.46, 0.46]
Cluster 4 :-
['Tiles', 'Tatched', 'R.C.C', 'Sheets', 'Stones']
[57.58, 38.48, 1.97, 1.69, 0.28]
Cluster 5 :-
['Tiles', 'Tatched', 'R.C.C', 'Sheets', 'Stones']
[59.2, 31.71, 5.71, 3.17, 0.21]
Cluster 6 :-
['Tatched', 'Tiles', 'Sheets', 'R.C.C', 'Stones']
[85.59, 11.04, 2.03, 1.13, 0.23]
Cluster 7 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets']
[75.92, 22.25, 1.05, 0.79]
Cluster 8 :-
['Tatched', 'Tiles', 'Sheets', 'R.C.C']
[66.46, 25.0, 5.49, 3.05]
Cluster 9 :-
['Tatched', 'Tiles', 'Sheets', 'R.C.C', 'Stones']
[90.7, 5.23, 1.74, 1.74, 0.58]
House type wise summary for all clusters:


For house type Tiles the total number o

Based on Guarantor

In [151]:
print('Clusterwise guarantor for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].index.tolist())
        print(count)        
        
df1 = summary.groupby('GUARANTOR').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['GUARANTOR','CLIENT_CODE']].copy()


print('Gurantor based summary for all clusters:\n')
for gtype in summary['GUARANTOR'].unique().tolist():
    print('\nFor guarantor',gtype,'the total number of applications are-',df1[df1['GUARANTOR']==htype]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Guarantor':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Guarantor']==gtype]['count']*100/float(df1[df1['GUARANTOR']==gtype]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise guarantor for all clusters

Cluster 0 :-
['N']
[100.0]
Cluster 1 :-
['N']
[100.0]
Cluster 2 :-
['N']
[100.0]
Cluster 3 :-
['N']
[100.0]
Cluster 4 :-
['N']
[100.0]
Cluster 5 :-
['N']
[100.0]
Cluster 6 :-
['N']
[100.0]
Cluster 7 :-
['N']
[100.0]
Cluster 8 :-
['N']
[100.0]
Cluster 9 :-
['N']
[100.0]
Gurantor based summary for all clusters:


For guarantor N the total number of applications are- Series([], )
Cluster 0 has below %
0    10.121747
Name: count, dtype: float64
Cluster 1 has below %
0    14.179995
Name: count, dtype: float64
Cluster 2 has below %
0    8.570064
Name: count, dtype: float64
Cluster 3 has below %
0    15.683934
Name: count, dtype: float64
Cluster 4 has below %
0    8.498448
Name: count, dtype: float64
Cluster 5 has below %
0    11.291478
Name: count, dtype: float64
Cluster 6 has below %
0    10.599188
Name: count, dtype: float64
Cluster 7 has below %
0    9.119122
Name: count, dtype: float64
Cluster 8 has below %
0    7.830031
Name: count, dtype: float64

In [152]:
#Based on Co obligant

print('Clusterwise co obligant for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('CO_OBLIGANT').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['CO_OBLIGANT','CLIENT_CODE']].copy()


print('Guarantor based summary for all clusters:\n')
for otype in summary['CO_OBLIGANT'].unique().tolist():
    print('\nFor co obligant',otype,'the total number of applications are-',df1[df1['CO_OBLIGANT']==otype]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Obligant':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Obligant']==otype]['count']*100/float(df1[df1['CO_OBLIGANT']==otype]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise co obligant for all clusters

Cluster 0 :-
['N', 'O']
[99.29, 0.71]
Cluster 1 :-
['N', 'O']
[98.82, 1.18]
Cluster 2 :-
['N', 'O']
[98.61, 1.39]
Cluster 3 :-
['N', 'O']
[98.78, 1.22]
Cluster 4 :-
['N', 'O']
[99.72, 0.28]
Cluster 5 :-
['N', 'O']
[88.79, 11.21]
Cluster 6 :-
['N', 'O']
[98.87, 1.13]
Cluster 7 :-
['N']
[100.0]
Cluster 8 :-
['N', 'O']
[95.43, 4.57]
Cluster 9 :-
['N', 'O']
[97.67, 2.33]
Guarantor based summary for all clusters:


For co obligant N the total number of applications are-  4088
Cluster 0 has below %
0    10.298434
Name: count, dtype: float64
Cluster 1 has below %
0    14.3591
Name: count, dtype: float64
Cluster 2 has below %
0    8.659491
Name: count, dtype: float64
Cluster 3 has below %
0    15.875734
Name: count, dtype: float64
Cluster 4 has below %
0    8.683953
Name: count, dtype: float64
Cluster 5 has below %
0    10.273973
Name: count, dtype: float64
Cluster 6 has below %
0    10.738748
Name: count, dtype: float64
Cluster 7 has below %
0    9.34

In [153]:
#Based on Collateral

print('Clusterwise collateral for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('CO_LATERAL').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['CO_LATERAL','CLIENT_CODE']].copy()


print('\nCollateral based summary for all clusters:')
for otype in summary['CO_LATERAL'].unique().tolist():
    print('\nFor collateral',otype,'the total number of applications are-',df1[df1['CO_LATERAL']==otype]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Collateral':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Collateral']==otype]['count']*100/float(df1[df1['CO_LATERAL']==otype]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise collateral for all clusters

Cluster 0 :-
['N']
[100.0]
Cluster 1 :-
['N']
[100.0]
Cluster 2 :-
['N']
[100.0]
Cluster 3 :-
['N']
[100.0]
Cluster 4 :-
['N']
[100.0]
Cluster 5 :-
['N']
[100.0]
Cluster 6 :-
['N']
[100.0]
Cluster 7 :-
['N']
[100.0]
Cluster 8 :-
['N']
[100.0]
Cluster 9 :-
['N']
[100.0]

Collateral based summary for all clusters:

For collateral N the total number of applications are-  4189
Cluster 0 has below %
0    10.121747
Name: count, dtype: float64
Cluster 1 has below %
0    14.179995
Name: count, dtype: float64
Cluster 2 has below %
0    8.570064
Name: count, dtype: float64
Cluster 3 has below %
0    15.683934
Name: count, dtype: float64
Cluster 4 has below %
0    8.498448
Name: count, dtype: float64
Cluster 5 has below %
0    11.291478
Name: count, dtype: float64
Cluster 6 has below %
0    10.599188
Name: count, dtype: float64
Cluster 7 has below %
0    9.119122
Name: count, dtype: float64
Cluster 8 has below %
0    7.830031
Name: count, dtype: float64
Cl

In [154]:
#Based on Gender

print('Clusterwise gender for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['GENDER'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['GENDER'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('GENDER').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['GENDER','CLIENT_CODE']].copy()


print('\nGender based summary for all clusters:')
for otype in summary['GENDER'].unique().tolist():
    print('\nFor gender',otype,'the total number of applications are-',df1[df1['GENDER']==otype]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['GENDER'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['GENDER'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'gender':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['gender']==otype]['count']*100/float(df1[df1['GENDER']==otype]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise gender for all clusters

Cluster 0 :-
['F', 'M']
[55.42, 44.58]
Cluster 1 :-
['F', 'M']
[65.99, 34.01]
Cluster 2 :-
['M', 'F']
[75.77, 24.23]
Cluster 3 :-
['F', 'M']
[73.21, 26.79]
Cluster 4 :-
['F', 'M']
[84.83, 15.17]
Cluster 5 :-
['M', 'F']
[70.61, 29.39]
Cluster 6 :-
['F', 'M']
[84.46, 15.54]
Cluster 7 :-
['F', 'M']
[98.17, 1.83]
Cluster 8 :-
['M', 'F']
[68.6, 31.4]
Cluster 9 :-
['F', 'M']
[63.95, 36.05]

Gender based summary for all clusters:

For gender F the total number of applications are-  2599
Cluster 0 has below %
0    9.041939
Name: count, dtype: float64
Cluster 1 has below %
0    15.082724
Name: count, dtype: float64
Cluster 2 has below %
1    3.347441
Name: count, dtype: float64
Cluster 3 has below %
0    18.507118
Name: count, dtype: float64
Cluster 4 has below %
0    11.619854
Name: count, dtype: float64
Cluster 5 has below %
1    5.348211
Name: count, dtype: float64
Cluster 6 has below %
0    14.428626
Name: count, dtype: float64
Cluster 7 has below %
0   

In [155]:
#summary.head()

In [156]:
#Based on Accomodation

print('Clusterwise accomodation for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['CLIENT_CODE']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('ACCOMADATION').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['ACCOMADATION','CLIENT_CODE']].copy()


print('\nAccomodation based summary for all clusters:')
for otype in summary['ACCOMADATION'].unique().tolist():
    print('\nFor accomodation',otype,'the total number of applications are-',df1[df1['ACCOMADATION']==otype]['CLIENT_CODE'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'acco':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['acco']==otype]['count']*100/float(df1[df1['ACCOMADATION']==otype]['CLIENT_CODE'].to_string(index=None)))
            

Clusterwise accomodation for all clusters

Cluster 0 :-
['O', 'R']
[77.83, 22.17]
Cluster 1 :-
['R', 'O']
[92.93, 7.07]
Cluster 2 :-
['R', 'O']
[79.67, 20.33]
Cluster 3 :-
['O', 'R']
[96.35, 3.65]
Cluster 4 :-
['R', 'O']
[96.91, 3.09]
Cluster 5 :-
['R', 'O']
[97.46, 2.54]
Cluster 6 :-
['R', 'O']
[99.32, 0.68]
Cluster 7 :-
['O', 'R']
[96.07, 3.93]
Cluster 8 :-
['R', 'O']
[99.39, 0.61]
Cluster 9 :-
['R', 'O']
[95.35, 4.65]

Accomodation based summary for all clusters:

For accomodation R the total number of applications are-  2708
Cluster 0 has below %
1    3.471196
Name: count, dtype: float64
Cluster 1 has below %
0    20.384047
Name: count, dtype: float64
Cluster 2 has below %
0    10.5613
Name: count, dtype: float64
Cluster 3 has below %
1    0.886263
Name: count, dtype: float64
Cluster 4 has below %
0    12.74003
Name: count, dtype: float64
Cluster 5 has below %
0    17.023634
Name: count, dtype: float64
Cluster 6 has below %
0    16.285081
Name: count, dtype: float64
Cluster 7 has b

SUMMARY STATS FOR CONTINUOUS NUMERICAL VARIABLES: APPL_AMT, FAMILY_MEM, REGULAR, FESTIVE, AGE

In [157]:
summary.head()

,CLIENT_CODE,LOAN_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,...,GENDER,ACCOMADATION,Default,APPL_AMT_CAT,FAMILY_MEM_CAT,REGULAR_CAT,FESTIVE_CAT,AGE_CAT,cluster,distance
0,31006642,9935635,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,...,F,R,0,"(0, 10000]","(2, 5]","(800, 1000]","(2000, 200000]","(25, 35]",6,6
1,31006645,9946982,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,...,F,R,1,"(15000, 200000]","(5, 100]","(800, 1000]","(1500, 2000]","(25, 35]",6,6
2,31006646,9946986,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,...,F,R,1,"(15000, 200000]","(5, 100]","(800, 1000]","(1500, 2000]","(35, 45]",4,7
3,31006652,9946985,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,...,F,R,1,"(15000, 200000]","(5, 100]","(800, 1000]","(1500, 2000]","(25, 35]",6,4
4,31006653,9946983,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,...,F,R,1,"(15000, 200000]","(2, 5]","(0, 500]","(950, 1500]","(25, 35]",5,5


In [158]:
for i in range(0,n_clust):
    sliced = summary[summary['cluster']==i]
    print('Stats for cluster',i,':-')
    print('1. Application amount-','Mean:',round(sliced['APPL_AMT'].mean(),ndigits=2),',Min:',round(sliced['APPL_AMT'].min(),ndigits=2),',Max:',round(sliced['APPL_AMT'].max(),ndigits=2),',Median:',round(sliced['APPL_AMT'].median(),ndigits=2))
    print('2. Family members-','Mean:',round(sliced['FAMILY_MEM'].mean(),ndigits=2),',Min:',round(sliced['FAMILY_MEM'].min(),ndigits=2),',Max:',round(sliced['FAMILY_MEM'].max(),ndigits=2),',Median:',round(sliced['FAMILY_MEM'].median(),ndigits=2))
    print('3. Regular-','Mean:',round(sliced['REGULAR'].mean(),ndigits=2),',Min:',round(sliced['REGULAR'].min(),ndigits=2),',Max:',round(sliced['REGULAR'].max(),ndigits=2),',Median:',round(sliced['REGULAR'].median(),ndigits=2))
    print('4. Festive-','Mean:',round(sliced['FESTIVE'].mean(),ndigits=2),',Min:',round(sliced['FESTIVE'].min(),ndigits=2),',Max:',round(sliced['FESTIVE'].max(),ndigits=2),',Median:',round(sliced['FESTIVE'].median(),ndigits=2))
    print('5. Age-','Mean:',round(sliced['AGE'].mean(),ndigits=2),',Min:',round(sliced['AGE'].min(),ndigits=2),',Max:',round(sliced['AGE'].max(),ndigits=2),',Median:',round(sliced['AGE'].median(),ndigits=2))
    print('\n')

Stats for cluster 0 :-
1. Application amount- Mean: 14531.72 ,Min: 10000.0 ,Max: 20000.0 ,Median: 15000.0
2. Family members- Mean: 2.22 ,Min: 1 ,Max: 6 ,Median: 2.0
3. Regular- Mean: 738.33 ,Min: 50.0 ,Max: 6600.0 ,Median: 500.0
4. Festive- Mean: 1887.71 ,Min: 90.0 ,Max: 25000.0 ,Median: 950.0
5. Age- Mean: 32.46 ,Min: 18.0 ,Max: 54.0 ,Median: 30.0


Stats for cluster 1 :-
1. Application amount- Mean: 14829.8 ,Min: 5000.0 ,Max: 150000.0 ,Median: 15000.0
2. Family members- Mean: 4.57 ,Min: 1 ,Max: 10 ,Median: 5.0
3. Regular- Mean: 1449.83 ,Min: 300.0 ,Max: 7000.0 ,Median: 1050.0
4. Festive- Mean: 3211.11 ,Min: 500.0 ,Max: 17000.0 ,Median: 3000.0
5. Age- Mean: 33.63 ,Min: 18.0 ,Max: 55.0 ,Median: 33.0


Stats for cluster 2 :-
1. Application amount- Mean: 14658.77 ,Min: 8000.0 ,Max: 45000.0 ,Median: 15000.0
2. Family members- Mean: 3.82 ,Min: 1 ,Max: 7 ,Median: 4.0
3. Regular- Mean: 741.11 ,Min: 50.0 ,Max: 3000.0 ,Median: 600.0
4. Festive- Mean: 1663.65 ,Min: 100.0 ,Max: 10000.0 ,Median: 